In [1]:
!pip install SQLAlchemy

## **Capitulo 1 - Schema & Tipos**

#### **Criando a Engine**

In [2]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///mydb")

### **Objeto MetaData**

O Objeto MetaData funciona como um catalago do banco de dados, por onde podemos listar e acessar todas as tabelas. Ele funciona como o objeto que "Junta" todos os elementos do banco em um mesmo contexto para posteriormente fazermos a persistência.

In [3]:
from sqlalchemy import MetaData

metadata = MetaData()

### **Criando uma tabela**

In [4]:
from sqlalchemy import Table, Column, Integer, Numeric, String, Boolean, CheckConstraint

# Tabelas são criadas com o método Table
# O primeiro argumento é o nome da tabela, o segundo o nosso objeto metadata
# E por fim, os outros parametros são as estruturas da tabela, nesse caso, colunas

cookies = Table('cookies', metadata,
  Column('cookie_id', Integer(), primary_key=True),
  Column('cookie_name', String(50), index=True),
  Column('cookie_recipe_url', String(255)),
  Column('cookie_sku', String(55)),
  Column('quantity', Integer(), CheckConstraint('quantity >= 0')),
  Column('unit_cost', Numeric(12, 2))
)

- A primeira coluna foi definida como chave primaria
- A segunda coluna foi indexada para deixar as queries com essa coluna mais rapidas
- A ultima coluna é um valor numerico de alta precisão, onde (12,2) significa que o digito pode ter até 12 numeros, com 2 sendo decimais (Após a virgula)

### **Criando a segunda tabela com datetime**

In [5]:
from datetime import datetime
from sqlalchemy import DateTime

users = Table('users', metadata,
  Column('user_id', Integer(), primary_key=True),
  Column('username', String(15), nullable=False, unique=True),
  Column('email', String(255), nullable=False),
  Column('phone', String(20), nullable=False),
  Column('password', String(25), nullable=False),
  Column('created_on', DateTime(), default=datetime.now),
  Column('updated_at', DateTime(), default=datetime.now, onupdate=datetime.now)
)

- O motivo de usar **datetime.now** é para que cada registro novo receba a data atual, caso fosse a função **datetime.now()**, ele criaria com a data em que a tabela foi criada;
- Com **default = datetime.now** estamos sinalizando que caso uma data não seja informada, a tabela sera preenchida com o horario atual, usando o atributo now da classe 'datetime';
- **unique = true** garante que o campo possua um valor único na tabela;
- **onupdate = datetime.now** essa linha faz com que o valor da coluna 'updated_at' seja atualizada usando o mesmo atributo now de datetime, sempre que um valor no registro mude.

### **Chaves e Restrições**

In [6]:
from sqlalchemy import PrimaryKeyConstraint, UniqueConstraint, CheckConstraint

# Definindo chaves e restrições PÓS-CRIAÇÃO da tabela

PrimaryKeyConstraint('user_id', name='user_pk')
UniqueConstraint('username', name='uix_username')

UniqueConstraint()

**CheckConstraint  -->** Usada para cada coluna para verificar se uma condição é verdadeira na hora de salvar um registro

In [7]:
CheckConstraint('unit_cost >= 0.00', name='unit_cost_positive')

CheckConstraint(<sqlalchemy.sql.elements.TextClause object at 0x7cc52a18cbc0>, name='unit_cost_positive')

### **Index**

um index serve para tornar uma consulta a uma certa coluna mais performática.<br>
Assim como as chaves e restrições, é possível criar um index pós criação da tabela, usando o Index do SQLAlchemy

In [8]:
from sqlalchemy import Index

Index('ix_cookies_cookie_name', 'cookie_name')

Index('ix_cookies_cookie_name', 'cookie_name')

Também é possível definir um index para uma consulta join

### **Relacionamentos e Foreign Key (+2 tabelas)**

In [9]:
from sqlalchemy import ForeignKey, ForeignKeyConstraint

orders = Table('orders', metadata,
  Column('order_id', Integer(), primary_key=True),
  Column('user_id', ForeignKey('users.user_id')),
  Column('shipped', Boolean(), default=False)
)

line_items = Table('line_items', metadata,
  Column('line_items_id', Integer(), primary_key=True),
  Column('order_id', ForeignKey('orders.order_id')),
  Column('cookie_id', ForeignKey('cookies.cookie_id')),
  Column('quantity', Integer()),
  Column('extended_cost', Numeric(12, 2))
)

Também é possível definir chaves estrangeiras explicitamente

In [10]:
# ForeignKeyConstrant(['order_id'],['orders.order_id'])

### **Persistencia das tabelas**

Para persistir a tabela, basta chamar o método **Create_all()** passando a engine. Assim, as tabelas serão salvas no banco. <br>
Não há risco de subscrição de tabelas que já existem, pois o método **NÃO deleta nem altera tabelas existentes**

In [11]:
metadata.create_all(engine)

## **Capitulo 2 - Trabalhando com Dados**

#### **Inserindo Dados**

Primeiro, fazemos a linha com a nossa query, nesse caso, usando o **insert().values(...)**

In [12]:
ins = cookies.insert().values(
    cookie_name="chocolate chip",
    cookie_recipe_url="http://some.aweso.me/cookie/recipe.html",
    cookie_sku="CC01",
    quantity="12",
    unit_cost="0.50"
)
print(str(ins))

INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (:cookie_name, :cookie_recipe_url, :cookie_sku, :quantity, :unit_cost)


Também é possível usar a função **insert()** passando a tabela como parametro, tornando a função mais versátil

In [13]:
from sqlalchemy import insert

ins = insert(cookies).values(
    cookie_name="chocolate chip",
    cookie_recipe_url="http://some.aweso.me/cookie/recipe.html",
    cookie_sku="CC01",
    quantity="12",
    unit_cost="0.50"
)
print(str(ins))

INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (:cookie_name, :cookie_recipe_url, :cookie_sku, :quantity, :unit_cost)


Esse código cria a query mas ainda não a executa. A query SQL é escrita com base no que informamos, usando uma forma parecida com **prepared statements**. Após a criação, vamos mandar a query compilar os valores informados

In [14]:
compiled = ins.compile()
compiled.params

{'cookie_name': 'chocolate chip',
 'cookie_recipe_url': 'http://some.aweso.me/cookie/recipe.html',
 'cookie_sku': 'CC01',
 'quantity': '12',
 'unit_cost': '0.50'}

Agora, vamos criar a conexão com a engine e executar nossa query compilada. Também é possível imprimir as chaves primarias que foram inseridas. <br>

a linha "**with engine.connect() as conn:**" serve para criar um bloco de código que irá criar uma conexão na engine, e após o fim do bloco de código, essa conexão é **fechada automaticamente**

In [15]:
result = None

with engine.connect() as conn:
  result = conn.execute(compiled)
  conn.commit()

In [16]:
result.inserted_primary_key

(1,)

Além de preparar uma versão compilada da query e seus valores, também é possível informar os paramêtros de insersão **dentro da função execute()**. Não é comumente usado para inserts únicos, mas pode ser util quando **queremos inserir varios registros**

In [17]:
"""
EXEMPLO: PASSANDO PARAMETROS VIA EXECUTE

ins = cookies.insert()
result = conn.execute(
  ins,
  cookie_name='dark chocolate chip'
  cookie_recipe_url="..."
  ...
)
result.inserted_primary_key

Aqui vemos que via execute() é possivel informar primeiro o tipo de query e depois passar os valores como parametros adicionais
"""

'\nEXEMPLO: PASSANDO PARAMETROS VIA EXECUTE\n\nins = cookies.insert()\nresult = conn.execute(\n  ins,\n  cookie_name=\'dark chocolate chip\'\n  cookie_recipe_url="..."\n  ...\n)\nresult.inserted_primary_key\n\nAqui vemos que via execute() é possivel informar primeiro o tipo de query e depois passar os valores como parametros adicionais\n'

In [18]:
# Inserindo uma lista de cookies (dicts)
inventory_list = [
  {
      'cookie_name': 'dark chocolate chip',
      'cookie_recipe_url': 'http://some.aweso.me/cookie/dark-chocolate.html',
      'cookie_sku': 'DC01',
      'quantity': '10',
      'unit_cost': '0.50'
  },
  {
      'cookie_name': 'peanut butter',
      'cookie_recipe_url': 'http://some.aweso.me/cookie/peanut.html',
      'cookie_sku': 'PB01',
      'quantity': '24',
      'unit_cost': '0.25'
  },
  {
      'cookie_name': 'oatmeal raisin',
      'cookie_recipe_url': 'http://some.aweso.me/cookie/raisin.html',
      'cookie_sku': 'EWW01',
      'quantity': '100',
      'unit_cost': '1.00'
  }
]

# preparando a query
ins = insert(cookies)

result = None
with engine.connect() as conn:
  result = conn.execute(ins, inventory_list)
  conn.commit()

#### **Consultando tabelas**

In [19]:
from sqlalchemy import select
s = select(cookies)

result = None
with engine.connect() as conn:
  res = conn.execute(s)
  print(f'{res.keys()}\n')  # imprimindo os nomes das colunas as colunas
  resultProxy = res
  result = res.fetchall()

display(type(result), result)

RMKeyView(['cookie_id', 'cookie_name', 'cookie_recipe_url', 'cookie_sku', 'quantity', 'unit_cost'])



list

[(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 12, Decimal('0.50')),
 (2, 'dark chocolate chip', 'http://some.aweso.me/cookie/dark-chocolate.html', 'DC01', 10, Decimal('0.50')),
 (3, 'peanut butter', 'http://some.aweso.me/cookie/peanut.html', 'PB01', 24, Decimal('0.25')),
 (4, 'oatmeal raisin', 'http://some.aweso.me/cookie/raisin.html', 'EWW01', 100, Decimal('1.00'))]

- a variavel "res" guarda um objeto do tipo ResultProxy, que guarda em um atributo o resultado da query e que podemos acessar via **fetchall()**. Similar a uma requisição rest.

- **res.keys()** ~> retorna uma lista de nomes de todas as colunas presentes nos registros

#### **ResultProxy**

O ResultProxy é um wrapper do objeto cursor DBAPI. Ele serve paraq **facilitar o uso e manipulação dos resultados de statements**. Algumas funções: <br>

- Permitir acesso aos registros (linhas) via index;
- Acessar coluna de um registro via index;
- Acessar coluna de um registro via nome;
- ~Acessar coluna de um registro usando o objeto Column~. **PESQUISAR MELHOR**

In [20]:
first_row = result[0]                    # selecionando a primeira linha dos resultados

print(first_row[1])                      # selecionando a primeira coluna (cookie_name) da primeira linha

print(first_row.cookie_name)             # selecionando o cookie_name via atributo da linha

chocolate chip
chocolate chip


Também é possível usar o ResultProxy para **iterar todas as linhas do resultado e fazer operações nessas linhas**

In [21]:
for record in result:
  print(record.cookie_name)

chocolate chip
dark chocolate chip
peanut butter
oatmeal raisin


Algumas outras funcionalidades do ResultProxy: <br>
- **first()** ~> retorna o primeiro registro caso exista e fecha a conexão;
- **fetchone()** ~> retorna apenas um registro mas mantem o cursor aberto para mais operações;
- **scalar()** ~> retorna um único valor CASO o resultado da query seja apenas um único registro com uma única coluna

Recomendações para um código melhor: <br>

- Usar **first** ao invés de *fetchone* ou *scalar*;
- Usar uma **iteração no ResultProxy** ao invés de *fetchall* e *fetchone*, pois é uma maneira mais performática e faz mais sentido ao trabalhar com uma linha por vez;
- Evitar **fetchone** pois pode deixar a conexão aberta em caso de descuido;
- Usar o método **scalar** esporadicamente, pois em caso de queries que retorna mais de 1 linha seu uso resulta em um erro.

#### **Escolhendo colunas na query**

É possível informar para a query que **só queremos algumas colunas** da tabela.

In [22]:
s = select(cookies.c.cookie_name, cookies.c.quantity)     # selecionando apenas 2 colunas

result = None
with engine.connect() as conn:
  res_s = conn.execute(s)
  print(f'KEYS: {res_s.keys()}\n')

  result = res_s.first()    # Retorna apenas o primeiro registro e fecha a conn

print(result)

KEYS: RMKeyView(['cookie_name', 'quantity'])

('chocolate chip', 12)


#### **Ordenação**

In [23]:
s = select(cookies.c.cookie_name, cookies.c.quantity)
s = s.order_by(cookies.c.quantity.desc())

with engine.connect() as conn:
  res_s = conn.execute(s)
  for cookie in res_s:
    print(f'{cookie.quantity} - {cookie.cookie_name}')

100 - oatmeal raisin
24 - peanut butter
12 - chocolate chip
10 - dark chocolate chip


#### **Limitando resultados**

In [24]:
s = select(cookies.c.cookie_name, cookies.c.quantity)
s = s.order_by(cookies.c.quantity)
s = s.limit(2)

with engine.connect() as conn:
  res_s = conn.execute(s)
  print([x.cookie_name for x in res_s])     # usando list comprehension

['dark chocolate chip', 'chocolate chip']


#### **Funções SQL e labels built-in**

Primeiro vamos fazer a função de agregação SUM()

In [25]:
from sqlalchemy import func

# Usamos o modulo func que traz algumas funções de agregações do SQL, como SUM() e COUNT()

s = select(func.sum(cookies.c.quantity))

with engine.connect() as conn:
  res_s = conn.execute(s)
  print(f'Quant. Total de Cookies: {res_s.scalar()}')


Quant. Total de Cookies: 146


Agora, vamos testar a função COUNT()

In [26]:
s = select(func.count(cookies.c.cookie_name))

with engine.connect() as conn:
  res_s = conn.execute(s)
  print(f'Nome da coluna: {res_s.keys()}')
  print(f'Contagem de registros: {res_s.scalar()}')


Nome da coluna: RMKeyView(['count_1'])
Contagem de registros: 4


Como podemos ver, a coluna gerada pela função COUNT() retorna com o nome genérico "count_1". Para mudar isso, podemos usar labels na query

In [27]:
s = select(func.count(cookies.c.cookie_name).label('inventory_count'))    # aplicando a função label() pós count()

with engine.connect() as conn:
  res_s = conn.execute(s)
  print(f'Nome da coluna: {res_s.keys()}')
  print(f'Contagem de registros: {res_s.scalar()}')

Nome da coluna: RMKeyView(['inventory_count'])
Contagem de registros: 4


#### **Aplicando filtros com where**

#### **Where puro + Conversão row em dict**

Além de aplicar o filtro where, tambem vamos aproveitar para ver como converter uma linha para dict, usando o método **_mapping** e depois um casting para dict

In [28]:
s = (select(cookies)
      .where(cookies.c.cookie_name == 'chocolate chip'))

with engine.connect() as conn:
  res_s = conn.execute(s)
  registro = res_s.first()
  print(f'{type(registro._mapping)}\n'\
        f'{registro._mapping}\n'\
        f'{dict(registro._mapping)}\n'\
        f'{type(dict(registro._mapping))}')

<class 'sqlalchemy.engine.row.RowMapping'>
{'cookie_id': 1, 'cookie_name': 'chocolate chip', 'cookie_recipe_url': 'http://some.aweso.me/cookie/recipe.html', 'cookie_sku': 'CC01', 'quantity': 12, 'unit_cost': Decimal('0.50')}
{'cookie_id': 1, 'cookie_name': 'chocolate chip', 'cookie_recipe_url': 'http://some.aweso.me/cookie/recipe.html', 'cookie_sku': 'CC01', 'quantity': 12, 'unit_cost': Decimal('0.50')}
<class 'dict'>


#### **Where aplicado o comando Like para busca em strings**

In [29]:
s = (select(cookies)
      .where(cookies.c.cookie_name.like('%chocolate%')))    # buscando todos os cookies com 'chocolate' no nome

with engine.connect() as conn:
  res_s = conn.execute(s)
  [print(x) for x in res_s]

(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 12, Decimal('0.50'))
(2, 'dark chocolate chip', 'http://some.aweso.me/cookie/dark-chocolate.html', 'DC01', 10, Decimal('0.50'))


#### **ClauseElements**

ClauseElements são uma entidade que usamos em uma clausula (Query?), e **costumam ser colunas em uma tabela**. Como exemplo temos o metodo .like() na coluna 'cookie_name' da tabela que usamos anteriormente. Existem alguns outros métodos úteis: <br>

Método                      | Utilidade
----------------------------|------------
between(*cleft*, *cright*)  | Acha uma coluna que esteja entre outras duas
concat(*column_two*)        | Concatena a coluna com a *column_two*
distinct()                  | Retorna apenas valores diferentes da coluna
in_(list)                   | Retorna apenas valores que estão na lista
is_(None)                   | Retorna apenas valores que são None (nulos)
contains(*String*)          | Retorna apenas valores que contem a *String* (case sensitive)
endswith(*String*)          | Retorna onde a coluna termina em *String* (case sensitive)
like(*String*)              | Retorna valores que possuem a *String* (case sensitive)
ilike(*String*)             | Igual o like(*String*), porem não é case sensitive)


#### **Operadores Aritmeticos**

Até agora usamos apenas o operador de igualdade (==) na clausula Where, mas podemos usar outros operadores lógicos.

Vamos fazer uma **concatenção** por exemplo:

In [30]:
s = select(cookies.c.cookie_name, 'SKU-'+cookies.c.cookie_sku)

with engine.connect() as conn:
  res_s = conn.execute(s)
  [print(x) for x in res_s]

('chocolate chip', 'SKU-CC01')
('dark chocolate chip', 'SKU-DC01')
('peanut butter', 'SKU-PB01')
('oatmeal raisin', 'SKU-EWW01')


Também podemos usar esses operadores para **realizar operações com o valor da coluna**, usando a função **cast** como forma de formatação do numero alem de conversão

In [31]:
from sqlalchemy import cast

s = (select(
      cookies.c.cookie_name,
      cast((cookies.c.quantity * cookies.c.unit_cost), Numeric(12,2)).label('inv_cost')
    )
)

with engine.connect() as conn:
  res_s = conn.execute(s)
  for row in res_s:
    print(f'Cookie: {row.cookie_name:<25} - Total do inventário: ${row.inv_cost}')



Cookie: chocolate chip            - Total do inventário: $6.00
Cookie: dark chocolate chip       - Total do inventário: $5.00
Cookie: peanut butter             - Total do inventário: $6.00
Cookie: oatmeal raisin            - Total do inventário: $100.00


<p> Veja que fizemos a <b>multiplicação</b> entre as colunas "quantidade" e "valor unitario" que resulta no valor total de cookies, por tipo de cookie. Tambem usamos novamente o método <b>label()</b> para renomear essa coluna calculada</p>
<p> A função <b>cast()</b> permite a conversão de tipos, nesse exemplo, os valores de 6.00 seriam algo como 6.00000000000, mas com a formatação para Numeric(12,2) ele voltou a ter apenas duas casas decimais. </p>
<p>Caso essa conversão seja apenas para exibição, <b>outra maneira de fazer isso é com fstrings e :.2f:</b></p>

#### **Operadores Lógicos e Booleanos**

O SQLAlchemy também tem acesso a operadores AND, OR e NOT em forma de operadores (&, |, ~). Entretanto, é preciso ter cuidado com a **ordem de execução desses operadores na linguagem python**, onde esses operadores tem prioridade acima dos *Operadores lógicos*, por exemplo: A < B & C < D, resulta na seguinte ordem: A < (B&C) < D. Quando o desejado seria (A < B) & (C < D). Veja a necessidade do uso de parentêses para alterar a ordem de execução.

Devido a essa natureza dos operadores booleanos, o ideal é utilizar **conjunções (Conjunctions)**

#### **Conjunções (Operadores Especiais)**

Apesar de ser possível unir duas clausulas Where() uma após a outra, utilizando conjunções podemos fazer a mesma coisa com uma legilibidade melhor.

In [32]:
from sqlalchemy import and_, or_, not_

s = (select(cookies).where(
      and_(
          cookies.c.quantity > 23,
          cookies.c.unit_cost < 0.40
      )
))

with engine.connect() as conn:
  res_s = conn.execute(s)
  for row in res_s:
    print(f'Cookie: {row.cookie_name:<20} | Qnt.: {row.quantity:<3} | Preço (Un): {row.unit_cost}')

Cookie: peanut butter        | Qnt.: 24  | Preço (Un): 0.25


Nesse exemplo, vimos o uso do **and_()** para aplicar duas condições ao Where(), nesse caso, cookies em que a quantidade for maior que 23 e o preço menor que 40 centavos.

In [33]:
s = (select(cookies).where(
      or_(
          cookies.c.quantity.between(10, 50),
          cookies.c.cookie_name.contains('chip')
      )
))

with engine.connect() as conn:
  res_s = conn.execute(s)
  for row in res_s:
    print(f'Cookie: {row.cookie_name:<20} | Qnt.: {row.quantity:<4}')

Cookie: chocolate chip       | Qnt.: 12  
Cookie: dark chocolate chip  | Qnt.: 10  
Cookie: peanut butter        | Qnt.: 24  


Agora, usamos o operador **or_()** para aplicar outra lógica: dessa vez, registro será selecionado caso uma das duas condições do Where() seja satisfeita

#### **Atualizando Dados**

Para fazer o update, como podemos prever, usamos o comando **update()** seguido do comando **Where()** para definir qual(is) valor(es) queremos atualizar. Caso executemos sem o Where, *Iremos atualizar todos os dados da tabela*

In [34]:
from sqlalchemy import update

u = update(cookies).where(cookies.c.cookie_name == "chocolate chip")
u = u.values(quantity = (cookies.c.quantity + 120))

with engine.connect() as conn:
  res_u = conn.execute(u)
  conn.commit()
  print(f'Linhas Alt. : {res_u.rowcount}')

Linhas Alt. : 1


In [35]:
# Vamos verificar se o update funcionou

s = select(cookies.c.cookie_name, cookies.c.quantity)
s = s.where(cookies.c.cookie_name == "chocolate chip")

with engine.connect() as conn:
  res_s = conn.execute(s)
  for row in res_s:
    print(f'{row.cookie_name} | {row.quantity}')

chocolate chip | 132


#### **Deletando dados**

Usamos novamente a lógica do SQL, dessa vez com o comando **delete()** com **Where()**

In [36]:
from sqlalchemy import delete

d = delete(cookies).where(cookies.c.cookie_name == 'dark chocolate chip')

with engine.connect() as conn:
  res_d = conn.execute(d)
  conn.commit()
  print(f'Linhas Alteradas: {res_d.rowcount}')

Linhas Alteradas: 1


In [37]:
s = select(cookies)
s = s.where(cookies.c.cookie_name == "dark chocolate chip")

with engine.connect() as conn:
  res_s = conn.execute(s)
  print(len(res_s.fetchall()))

0


#### **Populando as tabelas Users, Orders e Line_Items**

In [38]:
s = select(cookies)

with engine.connect() as conn:
  res_s = conn.execute(s)
  [print(row) for row in res_s]

(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 132, Decimal('0.50'))
(3, 'peanut butter', 'http://some.aweso.me/cookie/peanut.html', 'PB01', 24, Decimal('0.25'))
(4, 'oatmeal raisin', 'http://some.aweso.me/cookie/raisin.html', 'EWW01', 100, Decimal('1.00'))


Primeiro, vamos adicionar alguns clientes, mais precisamente, três clientes:

In [39]:
customer_list = [
    {
        'username': 'cookiemon',
        'email': 'mon@cookie.com',
        'phone': '111-111-1111',
        'password': 'password'
    },
    {
        'username': 'cakeeater',
        'email': 'cakeeater@cake.com',
        'phone': '222-222-2222',
        'password': 'password'
    },
    {
        'username': 'pieguy',
        'email': 'guy@pie.com',
        'phone': '333-333-333',
        'password': 'password'
    }
]

ins_users = users.insert()
with engine.connect() as conn:
  res_ins = conn.execute(ins_users, customer_list)
  conn.commit()
  print(f'Linhas alteradas: {res_ins.rowcount}')

Linhas alteradas: 3


Agora podemos preencher nossa **tabela relacionamento chamada 'orders'**. Vamos coemçar criando duas 'orders', e preencher a tabela 'line_items' com os itens de cada 'order'

In [40]:
# Inserindo as chaves estrangeiras na order
ins_order = insert(orders).values(user_id=1, order_id=1)

# criando os items do pedido (order)
order_items = [
    {
        "order_id": 1,
        "cookie_id": 1,
        "quantity": 2,
        "extended_cost": 1.00
    },
    {
        "order_id": 1,
        "cookie_id": 3,
        "quantity": 12,
        "extended_cost": 3.00
    }
]

ins_items = insert(line_items)

with engine.connect() as conn:
  res_ins_order = conn.execute(ins_order)
  print(f'Linhas inseridas (Orders): {res_ins_order.rowcount}')
  res_ins_items = conn.execute(ins_items, order_items)
  print(f'Linhas inseridas (Line_items): {res_ins_items.rowcount}')
  conn.commit()

Linhas inseridas (Orders): 1
Linhas inseridas (Line_items): 2


Agora vamos inserir estruturas similares para o cliente de id 2

In [41]:
# Inserindo as chaves estrangeiras na order
ins_order = insert(orders).values(user_id=2, order_id=2)

# criando os items do pedido (order)
order_items = [
    {
        "order_id": 2,
        "cookie_id": 1,
        "quantity": 24,
        "extended_cost": 12.00
    },
    {
        "order_id": 2,
        "cookie_id": 4,
        "quantity": 6,
        "extended_cost": 6.00
    }
]

ins_items = insert(line_items)

with engine.connect() as conn:
  res_ins_order = conn.execute(ins_order)
  print(f'Linhas inseridas (Orders): {res_ins_order.rowcount}')
  res_ins_items = conn.execute(ins_items, order_items)
  print(f'Linhas inseridas (Line_items): {res_ins_items.rowcount}')
  conn.commit()

Linhas inseridas (Orders): 1
Linhas inseridas (Line_items): 2


#### **Joins**

In [42]:
s = select(users)

with engine.connect() as conn:
  res_s = conn.execute(s)
  for row in res_s:
    print(row)

(1, 'cookiemon', 'mon@cookie.com', '111-111-1111', 'password', datetime.datetime(2025, 8, 20, 21, 42, 54, 699085), datetime.datetime(2025, 8, 20, 21, 42, 54, 699091))
(2, 'cakeeater', 'cakeeater@cake.com', '222-222-2222', 'password', datetime.datetime(2025, 8, 20, 21, 42, 54, 699093), datetime.datetime(2025, 8, 20, 21, 42, 54, 699094))
(3, 'pieguy', 'guy@pie.com', '333-333-333', 'password', datetime.datetime(2025, 8, 20, 21, 42, 54, 699095), datetime.datetime(2025, 8, 20, 21, 42, 54, 699096))


<p> Agora vamos aplicar join() e outerjoin() para consolidar dados relacionados de diferentes tabelas </p>
<p> Por exemplo, para cumprir o pedido do cliente 'cookiemon', precisamos determinar a quantidade de cada cookie do pedido. Isso requer o uso de três joins para conseguir todos os dados até o nome do cookie. </p>
<p> Dependendo de como os joins forem feitos, pode <b>ser necessário reposicionar a clausula 'FROM'.</b> Isso pois podemos querer selecionar apenas algumas colunas, o que será passado para o <b>select()</b>, mas depois ainda precisamos informar um 'FROM' para fazer os JOINs. Uma forma de fazer isso é com o comando <b>select_from()</b> do SQLAlchemy</p>

In [43]:
# escolhendo quais colunas queremos
columns = [orders.c.order_id,
           users.c.username,
           users.c.phone,
           cookies.c.cookie_name,
           line_items.c.quantity,
           line_items.c.extended_cost]

# estamos informando quais colunas queremos selecionar dentre as tabelas. Estamos desmontando o array com *args
cmon_orders = select(*columns)

# agora fazemos o select_from que ira conectar a tabela 'orders' através de JOINS com as outras 3 tabelas
cmon_orders = cmon_orders.select_from(orders.join(users).join(
                  line_items).join(cookies)).where(users.c.username == 'cookiemon')

print(f'{cmon_orders}\n')
result = None
with engine.connect() as conn:
  res = conn.execute(cmon_orders).fetchall()
  result = res
  for row in res:
    print(row)

SELECT orders.order_id, users.username, users.phone, cookies.cookie_name, line_items.quantity, line_items.extended_cost 
FROM orders JOIN users ON users.user_id = orders.user_id JOIN line_items ON orders.order_id = line_items.order_id JOIN cookies ON cookies.cookie_id = line_items.cookie_id 
WHERE users.username = :username_1

(1, 'cookiemon', '111-111-1111', 'chocolate chip', 2, Decimal('1.00'))
(1, 'cookiemon', '111-111-1111', 'peanut butter', 12, Decimal('3.00'))


Também será util obter uma **contagem de pedidos de todos os clientes**, incluindo os clientes que não possuem um pedido feito ainda. Para isso, vamos fazer um **outerjoin()**. Isso exige uma construção mais cuidadosa com a ordem dos joins, pois **a tabela na qual usarmos o outerjoin() será a que retornará os dados**

In [44]:
# vamos usar o objeto 'func' para usar a função de agregação COUNT
columns = [users.c.username, func.count(orders.c.order_id).label('count')]

all_orders = select(*columns)
all_orders = all_orders.select_from(users.outerjoin(orders))  # O SQLAlchemy sabe fazer o join por conta da chave estrangeira
all_orders2 = all_orders
all_orders = all_orders.group_by(users.c.username)

result = None
result2 = None
with engine.connect() as conn:
  res = conn.execute(all_orders).fetchall()
  result = res
  res_2 = conn.execute(all_orders2).fetchall()
  result2 = res_2

print(f'Cliente       | Qnt. Pedidos')
print(f'----------------------------')
for row in result:
  print(f'{row.username:<14}| {row.count:<3}')

print('\nÉ isso que acontece sem a cláusula "groupby"')
for row in result2:
  print(row)

Cliente       | Qnt. Pedidos
----------------------------
cakeeater     | 1  
cookiemon     | 1  
pieguy        | 0  

É isso que acontece sem a cláusula "groupby"
('cakeeater', 2)


Como podemos ver, é necessário uma cláusula **group_by()** para que haja a separação da contagem de 'order_id' para cada 'username'

#### **Grouping**

Para aplicar agrupamentos, é necessário ter uma ou mais colunas para serem agrupadas e uma ou mais colunas que deem sentido ao agrupamento com funções de agregações como counts, sums, etc. Como feito em SQL padrão. Como vimos no ultimo código, selecionamos duas colunas (username, contagem de order_id), sendo a segunda uma **função de agregação COUNT**, que nos permite usá-la para um agrupamento de baseando em outra coluna (username)

#### **Chaining**

<p> Nós ja usamos chaining nos exemplos anteriores, mas sem explicitá-lo. </p>
<p> Chaining consiste em aplicar funções seguidas para obter um resultado, como fazemos com as queries quando usamos JOIN, ou um simples WHERE </p>
<p> Vamos <b>criar uma função para retornar uma lista de pedidos por cliente: </b></p>

In [45]:
def get_orders_by_customer(cust_name:str):
  columns = [orders.c.order_id,
            users.c.username,
            users.c.phone,
            cookies.c.cookie_name,
            line_items.c.quantity,
            line_items.c.extended_cost]

  cust_orders = select(*columns)

  cust_orders = cust_orders.select_from(orders.join(users).join(
                    line_items).join(cookies)).where(users.c.username == cust_name)

  result = None
  with engine.connect() as conn:
    result = conn.execute(cust_orders).fetchall()

  return result

In [46]:
get_orders_by_customer('cakeeater')

[(2, 'cakeeater', '222-222-2222', 'chocolate chip', 24, Decimal('12.00')),
 (2, 'cakeeater', '222-222-2222', 'oatmeal raisin', 6, Decimal('6.00'))]

<p> Porém, e se quisermos ver apenas os pedidos que já foram enviados, mas sem perder a funcionalidade atual?</p>
<p> Para isso, podemos <b>usar condicionais para construir nossa função e a sequencia de chaining</b>. Isso nos permite controlar ainda mais a query. Vamos aplicar duas condicionais, uma para se o pedido foi entregue, e outra para ver os detalhes do pedido.</p>

In [47]:
def get_orders_by_customer_2(cust_name:str, shipped=None, details=False):
  columns = [orders.c.order_id,
             users.c.username,
             users.c.phone]

  joins = users.join(orders)

  # alterando as colunas e joins dependendo da condicional
  if details:
    columns.extend([cookies.c.cookie_name,
                    line_items.c.quantity,
                    line_items.c.extended_cost])
    joins = joins.join(line_items).join(cookies)

  cust_orders = select(*columns)
  cust_orders = cust_orders.select_from(joins)
  cust_orders = cust_orders.where(users.c.username == cust_name)

  # aplicando outro filtro caso a coluna 'shipped',
  if shipped is not None:
    cust_orders = cust_orders.where(orders.c.shipped == shipped)

  result = None
  with engine.connect() as conn:
    result = conn.execute(cust_orders).fetchall()

  return result

Agora vamos testar essa nova função:

In [48]:
get_orders_by_customer_2('cakeeater')

[(2, 'cakeeater', '222-222-2222')]

In [49]:
get_orders_by_customer_2('cakeeater', details=True)

[(2, 'cakeeater', '222-222-2222', 'chocolate chip', 24, Decimal('12.00')),
 (2, 'cakeeater', '222-222-2222', 'oatmeal raisin', 6, Decimal('6.00'))]

In [50]:
get_orders_by_customer_2('cakeeater', shipped=True)

[]

In [51]:
get_orders_by_customer_2('cakeeater', shipped=False)

[(2, 'cakeeater', '222-222-2222')]

In [52]:
get_orders_by_customer_2('cakeeater', shipped=False, details=True)

[(2, 'cakeeater', '222-222-2222', 'chocolate chip', 24, Decimal('12.00')),
 (2, 'cakeeater', '222-222-2222', 'oatmeal raisin', 6, Decimal('6.00'))]

#### **Queries Cruas (Raw)**

Também é possível rodar queries em SQL cru. Fazendo isso, ainda recebemos um **ResultProxy** que ainda pode ser manipulado (chaining, etc.) assim como uma query do SQLAlchemy.

In [53]:
from sqlalchemy import text

with engine.connect() as conn:
  res = conn.execute(text("SELECT * FROM orders")).fetchall()
  for row in res:
    print(row)

(1, 1, 0)
(2, 2, 0)


Também podemos misturar partes de SQL cru com a sintaxe do SQLAlchemy

In [54]:
with engine.connect() as conn:
  res = conn.execute(
            select(users).where(text("username='cookiemon'"))
                ).fetchall()

  for row in res:
    print(row)

(1, 'cookiemon', 'mon@cookie.com', '111-111-1111', 'password', datetime.datetime(2025, 8, 20, 21, 42, 54, 699085), datetime.datetime(2025, 8, 20, 21, 42, 54, 699091))


## **Capitulo 3 - Exceptions & Transactions**

#### **Exceptions**

TODO

#### **Transactions**

Transações consistem em uma **sequência de queries feitas no banco dados**, Pórem, caso uma delas falhe, todas as outras são canceladas antes de ser feito um commit.

Vamos trabalhar com trasações nesse banco de dados que criamos. Um lugar onde podemos aplicar isso é nos pedidos de cookies, pois antes de enviar os pedidos **precisamos verificar se há cookies suficientes no estoque**<br>

Vamos criar uma função **ship_it(*order_id*)** que ficará responsável por essa verificação:

In [55]:
from sqlalchemy.exc import IntegrityError

def ship_it(order_id):
  s = select(line_items.c.cookie_id, line_items.c.quantity)
  s = s.where(line_items.c.order_id == order_id)

  result = None
  with engine.connect() as conn:
    result = conn.execute(s).fetchall()

  try:
    with engine.begin() as transaction:
      for cookie in result:
        u = update(cookies).where(cookies.c.cookie_id == cookie.cookie_id)
        u = u.values(quantity = cookies.c.quantity - cookie.quantity)
        transaction.execute(u)

      u = update(orders).where(orders.c.order_id == order_id)
      u = u.values(shipped = True)
      transaction.execute(u)
      print(f'Shipped Order ID: {order_id}')
  except IntegrityError as e:
    print(f'ERRO DE INTEGRIDADE: \n{e}')

def get_all_cookies_quantity():
  s = select(cookies.c.cookie_name, cookies.c.quantity)

  result = None
  with engine.connect() as conn:
    result = conn.execute(s).fetchall()

  return result

In [56]:
def get_order_items(order_id):
  s = select(line_items.c.order_id,
             line_items.c.line_items_id,
             cookies.c.cookie_name,
             line_items.c.quantity)

  s = s.select_from(line_items.join(cookies))
  s = s.where(line_items.c.order_id == order_id)

  with engine.connect() as conn:
    result = conn.execute(s)
    [print(row) for row in result]

In [57]:
# imprimindo todas as
with engine.connect() as conn:
  result = conn.execute(select(line_items))
  for row in result:
    print(row)

(1, 1, 1, 2, Decimal('1.00'))
(2, 1, 3, 12, Decimal('3.00'))
(3, 2, 1, 24, Decimal('12.00'))
(4, 2, 4, 6, Decimal('6.00'))


Chamando a função para teste com o pedido 1

In [58]:
print(get_all_cookies_quantity())

# Aqui estamos testando com o pedido 1
ship_it(1)

print(get_all_cookies_quantity())

[('chocolate chip', 132), ('peanut butter', 24), ('oatmeal raisin', 100)]
Shipped Order ID: 1
[('chocolate chip', 130), ('peanut butter', 12), ('oatmeal raisin', 100)]


Como podemos ver, o pedido 1 era de 2 cookies 'chocolate chip' e 12 cookies 'peanut butter'. Agora, sobraram 12 cookies 'peanut butter' no estoque. Vamos adicionar um pedido novo que tenha como items 14 cookies 'peanut butter'.

In [59]:
ins_order = insert(orders).values(user_id=1, order_id=3)
ins_items = insert(line_items)

novo_pedido = {
    "order_id": 3,
    "cookie_id": 3,
    "quantity": 14,
    "extended_cost": 3.50
}

with engine.connect() as conn:
  conn.execute(ins_order)
  conn.execute(ins_items, novo_pedido)
  conn.commit()

Agora vamos ver os items do pedido que adicionamos:

In [60]:
get_order_items(3)

(3, 5, 'peanut butter', 14)


Vamos tentar enviá-lo, sabendo que só temos 12 cookies e o pedido pede 14

In [61]:
ship_it(3)

ERRO DE INTEGRIDADE: 
(sqlite3.IntegrityError) CHECK constraint failed: quantity >= 0
[SQL: UPDATE cookies SET quantity=(cookies.quantity - ?) WHERE cookies.cookie_id = ?]
[parameters: (14, 3)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


Podemos ver que deu um erro de integridade devido a nossa CheckConstraint que garante que a coluna 'quantity' seja sempre >= 0